# django-postgre-copy speed tests

By Ben Welsh

This notebook tests the effect of dropping database constraints and indexes prior to loading a large data file.

The official PostgreSQL documentation suggests it lead to significant gains.

We will test this claim by dropping constraints and indexes prior to loading data from the California Civic Data Coalition via the django-postgres-copy wrapper on the database's COPY command.

### Connect California Civic Data Coalition Django project

Import Python tools

In [1]:
import os
import sys

In [2]:
import warnings
warnings.simplefilter("ignore")

Add the Django settings module to the environment.

In [3]:
sys.path.insert(0, '/home/palewire/.virtualenvs//django-calaccess-raw-data/src/')
sys.path.insert(0, '/home/palewire/.virtualenvs//django-calaccess-raw-data/lib/python2.7/')
sys.path.insert(0, '/home/palewire/.virtualenvs//django-calaccess-raw-data/lib/python2.7/site-packages/')
sys.path.insert(0, '/home/palewire/Code/django-calaccess-raw-data/')
sys.path.insert(0, '/home/palewire/Code/django-calaccess-raw-data/example/')
sys.path.insert(0, '/home/palewire/Code/django-postgres-copy/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")

'settings'

Verify we have the correct version of django-postgres-copy

In [4]:
import postgres_copy

In [5]:
postgres_copy.__version__

'2.0.0'

Boot the Django project

In [6]:
%%capture
import django
django.setup()

### Speed tests

Import database models

In [7]:
from calaccess_raw import models 

Verify we can query a table and time results

In [9]:
%timeit -n 5 models.CvrRegistrationCd.objects.count()

5 loops, best of 3: 442 µs per loop


Prep a couple functions we'll use during loading.

In [10]:
def model_mapping(model):
     return dict(
        (f.name, f.db_column) for f in model._meta.fields
        if f.db_column
    )

In [11]:
def test_small(drop_constraints=True, drop_indexes=True):
    models.CvrRegistrationCd.objects.all().delete()
    models.CvrRegistrationCd.objects.from_csv(
        "./cvr_registration_cd.csv",
        model_mapping(models.CvrRegistrationCd),
        drop_constraints=drop_constraints,
        drop_indexes=drop_indexes
    )

Show how many rows are in the file we'll be loading

In [12]:
!wc -l ./cvr_registration_cd.csv

52025 ./cvr_registration_cd.csv


Test how long it takes to load data when the constraints stay

In [13]:
%timeit -n 10 test_small(drop_constraints=False, drop_indexes=False)

10 loops, best of 3: 3.42 s per loop


Test how long it takes to load data when constraints are dropped

In [14]:
%timeit -n 10 test_small(drop_constraints=True, drop_indexes=True)

10 loops, best of 3: 3.96 s per loop


In [18]:
def test_large(drop_constraints=True, drop_indexes=True):
    models.RcptCd.objects.all().delete()
    models.RcptCd.objects.from_csv(
        "./rcpt_cd.csv",
        model_mapping(models.RcptCd),
        drop_constraints=drop_constraints,
        drop_indexes=drop_indexes
    )

In [16]:
!wc -l ./rcpt_cd.csv

10342161 ./rcpt_cd.csv


In [19]:
%timeit -n 3 test_large(drop_constraints=False, drop_indexes=False)

3 loops, best of 3: 8min 3s per loop


In [20]:
%timeit -n 3 test_large(drop_constraints=True, drop_indexes=True)

3 loops, best of 3: 8min 47s per loop
